## Isolating the FastSpecFit fitting engine

This notebook basically breaks down the tasks in `fastspecfit.fastspec` so that the parallelized fitting engine, `fastspecfit.fastspec_one` can be called separately / independently.

In [1]:
import os
import numpy as np
from astropy.table import vstack
from desispec.io import findfile

In [2]:
from fastspecfit.singlecopy import sc_data
from fastspecfit.fastspecfit import fastspec_one
from fastspecfit.io import (DESISpectra, get_output_dtype, create_output_meta,
                            create_output_table, write_fastspecfit)

In [3]:
specprod = 'loa'
survey, program, healpix = 'main', 'dark', 2230

redrockfile = findfile('redrock', survey=survey, faprogram=program, healpix=healpix, groupname='healpix', 
                       specprod_dir=os.path.join(os.getenv('DESI_SPECTRO_REDUX'), specprod))
redrockfile

'/global/cfs/cdirs/desi/spectro/redux/loa/healpix/main/dark/22/2230/redrock-main-dark-2230.fits'

#### Location and filename for outputs

In [4]:
outdir = os.getenv('PSCRATCH')
outfile = os.path.join(outdir, f'fastspec-{survey}-{program}-{healpix}.fits')
outfile

'/pscratch/sd/i/ioannis/fastspec-main-dark-2230.fits'

#### Initialize / read the templates, emission-line table, photometry metadata, cosmology, and IGM tables just once.

In [5]:
sc_data.initialize(log_verbose=True)

DEBUG:singlecopy.py:47:initialize: Cached stellar templates /global/cfs/cdirs/desi/external/templates/fastspecfit/2.0.0/ftemplates-chabrier-2.0.0.fits
DEBUG:singlecopy.py:51:initialize: Cached emission-line table /global/common/software/desi/perlmutter/desiconda/20240425-2.2.0/code/fastspecfit/main/py/fastspecfit/data/emlines.ecsv
DEBUG:singlecopy.py:56:initialize: Cached photometric filters and parameters /global/common/software/desi/perlmutter/desiconda/20240425-2.2.0/code/fastspecfit/main/py/fastspecfit/data/legacysurvey-dr9.yaml
DEBUG:singlecopy.py:60:initialize: Cached cosmology table /global/common/software/desi/perlmutter/desiconda/20240425-2.2.0/code/fastspecfit/main/py/fastspecfit/data/desi_fiducial_cosmology.dat
DEBUG:singlecopy.py:64:initialize: Cached Inoue+2014 IGM attenuation parameters.


#### Read the spectra and metadata for the given healpixel

The `gather_metadata` method expects a list of redrockfiles (one per healpixels) and returns `data`, a list of dictionaries (one per healpixel) which hold the spectra and a few other quantities that we only want to compute once. It also returns `meta`, an astropy table (stacked over all input healpixels) which will be written out as the `METADATA` HDU.

NB: Set `ntargets=None` to process the whole healpixel.

In [6]:
ntargets = 2

In [7]:
Spec = DESISpectra(phot=sc_data.photometry, cosmo=sc_data.cosmology)
Spec.gather_metadata(np.atleast_1d(redrockfile), ntargets=ntargets)
data, meta = Spec.read(sc_data.photometry)

INFO:io.py:676:gather_metadata: specprod=loa, coadd_type=healpix, survey=main, program=dark, healpix=2230
INFO:io.py:1056:read: Reading 2 spectra from /global/cfs/cdirs/desi/spectro/redux/loa/healpix/main/dark/22/2230/coadd-main-dark-2230.fits


In [8]:
print(data[0].keys())

dict_keys(['uniqueid', 'redshift', 'photsys', 'cameras', 'dluminosity', 'dmodulus', 'tuniv', 'ebv', 'wave0', 'flux0', 'ivar0', 'mask0', 'res0', 'coadd_wave', 'coadd_flux', 'coadd_ivar', 'coadd_res', 'mw_transmission_fiberflux'])


In [9]:
print(len(meta), meta.colnames)

2 ['Z', 'ZWARN', 'SPECTYPE', 'SUBTYPE', 'DELTACHI2', 'Z_RR', 'ZWARN_RR', 'TARGETID', 'COADD_FIBERSTATUS', 'TARGET_RA', 'TARGET_DEC', 'OBJTYPE', 'RELEASE', 'BRICKNAME', 'BRICKID', 'BRICK_OBJID', 'PHOTSYS', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'SCND_TARGET', 'TSNR2_BGS', 'TSNR2_ELG', 'TSNR2_LRG', 'TSNR2_LYA', 'TSNR2_QSO', 'TILEID_LIST', 'SURVEY', 'PROGRAM', 'HEALPIX', 'DEC', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'FIBERTOTFLUX_G', 'FIBERTOTFLUX_R', 'FIBERTOTFLUX_Z', 'FLUX_G', 'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4', 'FLUX_IVAR_Z', 'FLUX_R', 'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4', 'FLUX_Z', 'LS_ID', 'RA', 'EBV', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4']


#### Prepare the output objects.

The data model is documented at https://fastspecfit.readthedocs.io/en/latest/fastspec.html

In [10]:
ntargets = len(meta)
ncoeff = sc_data.templates.ntemplates
cameras = data[0]['cameras']

fastfit_dtype, fastfit_units = get_output_dtype(
    Spec.specprod, phot=sc_data.photometry,
    linetable=sc_data.emlines.table, ncoeff=ncoeff,
    cameras=cameras)

specphot_dtype, specphot_units = get_output_dtype(
    Spec.specprod, phot=sc_data.photometry,
    linetable=sc_data.emlines.table, ncoeff=ncoeff,
    cameras=cameras, specphot=True)

#### Initialize the random-number generation so that Monte Carlo realizations are fully reproducible.

In [11]:
seed = 42
rng = np.random.default_rng(seed=seed)
seeds = rng.integers(2**32, size=ntargets, dtype=np.int64)

#### Fit serially.

In [12]:
out = []
for iobj in range(ntargets):
    out.append(fastspec_one(iobj, data[iobj], meta[iobj], fastfit_dtype, specphot_dtype, seed=seeds[iobj]))
out = list(zip(*out))

INFO:fastspecfit.py:92:fastspec_one: Continuum- and emission-line fitting object 0 [targetid 39628530157359087, seed 383329928, z=0.634246].
INFO:continuum.py:1242:continuum_fastspec: Median spectral S/N_b=0.56 S/N_r=3.81 S/N_z=6.08
INFO:continuum.py:1143:vdisp_by_chi2scan: Initial velocity dispersion fit succeeded: delta-chi2=181>25, vdisp_init=288 km/s
INFO:continuum.py:1433:continuum_fastspec: Median aperture correction 1.245 [1.058-1.263].
INFO:continuum.py:1630:continuum_specfit: Smooth continuum correction: b=-1.282% r=0.842% z=0.787%
INFO:continuum.py:1835:continuum_specfit: vdisp=286.99+/-32.22 km/s log(M/Msun)=10.96+/-0.01 tau(V)=0.08+/-0.04 Age=6.28+/-0.01 Gyr SFR=0.00+/-0.00 Msun/yr Z/Zsun=0.00
INFO:emlines.py:1591:emline_specfit: delta(v) UV=249.4 km/s Balmer broad=0.0 km/s narrow=-173.5 km/s
INFO:emlines.py:1601:emline_specfit: sigma UV=3051 km/s Balmer broad=0 km/s narrow=58 km/s
INFO:emlines.py:1632:emline_specfit: Dn(4000)=1.730 in the emission-line subtracted spectrum.

#### Parse the results and write out.

In [13]:
outmeta = create_output_meta(vstack(out[0]), phot=sc_data.photometry)
specphot = create_output_table(out[1], outmeta, specphot_units)
fastfit = create_output_table(out[2], outmeta, fastfit_units)
modelspectra = vstack(out[3], join_type='exact', metadata_conflicts='error')

write_fastspecfit(
    outmeta, specphot, fastfit, modelspectra=modelspectra,
    outfile=outfile, specprod=Spec.specprod, coadd_type=Spec.coadd_type,
    fphotofile=sc_data.photometry.fphotofile, template_file=sc_data.templates.file,
    emlinesfile=sc_data.emlines.file, seed=seed)

INFO:io.py:1667:write: Writing 2 objects to /pscratch/sd/i/ioannis/fastspec-main-dark-2230.fits
